# 🚀 產品推薦系統 - 快速開始（簡化版）

這是一個簡化版的 Colab Notebook，專注於快速展示核心功能。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ldsAS/product-recommendation-system/blob/main/notebooks/colab_quickstart.ipynb)

## 步驟 1: 環境設置

In [ ]:
%%capture
# 安裝依賴（隱藏輸出）
!pip install pandas numpy scikit-learn lightgbm pydantic pydantic-settings python-dotenv

In [ ]:
# 克隆專案
!git clone https://github.com/ldsAS/product-recommendation-system.git
%cd product-recommendation-system

# 設置環境
import os
import sys

project_path = '/content/product-recommendation-system'
os.chdir(project_path)
sys.path.insert(0, project_path)
os.environ['PYTHONPATH'] = project_path

print("✅ 環境設置完成！")

## 步驟 2: 生成示範資料

In [ ]:
# 生成示範資料
!python scripts/generate_demo_data.py --members 50 --products 30 --sales 200

# 驗證資料
import pandas as pd

members = pd.read_csv('data/raw/member')
sales = pd.read_csv('data/raw/sales')
salesdetails = pd.read_csv('data/raw/salesdetails')

print(f"\n✅ 資料載入成功！")
print(f"會員: {len(members)} 筆")
print(f"訂單: {len(sales)} 筆")
print(f"明細: {len(salesdetails)} 筆")

## 步驟 3: 訓練模型（簡化版）

In [ ]:
# 使用 Python 直接執行訓練（避免路徑問題）
print("🚀 開始訓練模型...\n")

# 確保在正確目錄
os.chdir('/content/product-recommendation-system')

# 直接執行訓練腳本
exec(open('src/train.py').read())

print("\n✅ 訓練完成！")

## 步驟 4: 測試推薦

In [ ]:
# 導入模組
from src.models.recommendation_engine import RecommendationEngine
from src.models.data_models import RecommendationRequest

# 初始化引擎
engine = RecommendationEngine()

# 選擇測試會員
test_member = members.iloc[0]

# 創建請求
request = RecommendationRequest(
    member_code=test_member['會員編號'],
    phone=test_member['電話'],
    total_consumption=float(test_member['總消費金額']),
    accumulated_bonus=float(test_member['累積紅利']),
    recent_purchases=[]
)

# 獲取推薦
print(f"🎯 為會員 {test_member['會員編號']} 生成推薦\n")
print(f"總消費: ${test_member['總消費金額']:,}")
print(f"累積紅利: {test_member['累積紅利']} 點\n")

recommendations = engine.get_recommendations(request)

print("✨ 推薦結果:")
print("=" * 50)

for i, rec in enumerate(recommendations.recommendations, 1):
    print(f"\n{i}. 產品 {rec.product_id}")
    print(f"   信心度: {rec.score:.1%}")
    print(f"   理由: {rec.reason}")

print("\n" + "=" * 50)
print(f"回應時間: {recommendations.response_time_ms:.0f} ms")
print("\n✅ 推薦成功！")

## 步驟 5: 批次測試

In [ ]:
# 測試多個會員
print("📊 批次測試 10 個會員...\n")

results = []

for idx in range(min(10, len(members))):
    member = members.iloc[idx]
    request = RecommendationRequest(
        member_code=member['會員編號'],
        phone=member['電話'],
        total_consumption=float(member['總消費金額']),
        accumulated_bonus=float(member['累積紅利']),
        recent_purchases=[]
    )
    
    try:
        result = engine.get_recommendations(request)
        avg_score = sum(r.score for r in result.recommendations) / len(result.recommendations)
        results.append({
            'member': member['會員編號'],
            'avg_score': avg_score,
            'response_time': result.response_time_ms
        })
        print(f"✓ {member['會員編號']}: 平均分數 {avg_score:.1%}, {result.response_time_ms:.0f}ms")
    except Exception as e:
        print(f"✗ {member['會員編號']}: 失敗 - {e}")

if results:
    import pandas as pd
    results_df = pd.DataFrame(results)
    
    print("\n" + "=" * 50)
    print("📈 統計結果:")
    print(f"平均信心分數: {results_df['avg_score'].mean():.1%}")
    print(f"平均回應時間: {results_df['response_time'].mean():.0f} ms")
    print(f"最大回應時間: {results_df['response_time'].max():.0f} ms")
    print("=" * 50)

## 🎉 完成！

您已成功在 Colab 上運行產品推薦系統！

### 下一步

- 上傳您自己的資料檔案
- 調整模型參數
- 查看完整版 Notebook: [colab_demo.ipynb](colab_demo.ipynb)
- 訪問 [GitHub 倉庫](https://github.com/ldsAS/product-recommendation-system)